In [1]:
import pandas as pd
import json

from vcub_keeper.reader.reader import read_activity_vcub
from vcub_keeper.production.data import transform_json_api_bdx_station_data_to_df


from vcub_keeper.config import ROOT_TESTS_DATA

%load_ext autoreload
%autoreload 2

Permets d'extraire les données non simulée afin de faire les tests de benchmark pour les fonctions suivantes :
- `get_transactions_out()`
- `get_transactions_in()`

In [47]:
# Lecture de l'activité des stations
activite = read_activity_vcub()

In [48]:
list_station_id = [106, 22, 43, 102, 123]
start_date = "2017-07-10" 
end_date = "2017-07-14"

export = activite[(activite['station_id'].isin(list_station_id)) & (activite['date'] >= start_date) & (activite['date'] <= end_date)].copy()
export.shape

(5630, 8)

In [49]:
export.station_id.unique()

array([ 22,  43, 102, 106, 123], dtype=uint8)

In [50]:
export.head()

,gid,station_id,type,name,state,available_stands,available_bikes,date
454307,92,22,VLS,Hotel de Ville,1,31,2,2017-07-10 00:04:04
454308,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:09:04
454309,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:14:05
454310,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:19:04
454311,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:24:05


In [51]:
export.dtypes

gid                          uint8
station_id                   uint8
type                      category
name                string[python]
state                     category
available_stands             uint8
available_bikes              uint8
date                datetime64[ns]
dtype: object

In [52]:
# have to change some columns to simulate original data


state_dict = {1: "CONNECTEE", 0: "DECONNECTEE"}
export["state"] = export["state"].map(state_dict)

export = export.rename(columns={"station_id": "ident",
                                "date": "ts"})

### Export des données de test

In [53]:
export.to_csv(ROOT_TESTS_DATA + "activite_data.csv", index=False)

In [54]:
result_read_df = read_activity_vcub(file_path=ROOT_TESTS_DATA + "activite_data.csv")

In [55]:
export_test = activite[(activite['station_id'].isin(list_station_id)) & (activite['date'] >= start_date) & (activite['date'] <= end_date)].copy()

In [56]:
pd.testing.assert_frame_equal(result_read_df.reset_index(drop=True), 
                              export_test.reset_index(drop=True), 
                              check_categorical=False, 
                              check_dtype=False)

In [57]:
result_read_df.reset_index(drop=True).head()

,gid,station_id,type,name,state,available_stands,available_bikes,date
0,92,22,VLS,Hotel de Ville,1,31,2,2017-07-10 00:04:04
1,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:09:04
2,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:14:05
3,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:19:04
4,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:24:05


In [58]:
export_test.reset_index(drop=True).head()

,gid,station_id,type,name,state,available_stands,available_bikes,date
0,92,22,VLS,Hotel de Ville,1,31,2,2017-07-10 00:04:04
1,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:09:04
2,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:14:05
3,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:19:04
4,92,22,VLS,Hotel de Ville,1,33,0,2017-07-10 00:24:05
